## Aprendizado Supervisionado
### prof. Duncan
Prática com KNN e diabetes data set

In [ ]:
#pacote para profiling de datasets
#!pip install ydata_profiling
# pacotes básicos
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from ydata_profiling import ProfileReport as PR

# pacotes do sklearn para acesso a datasets, preparação, modelagem e avaliação
from sklearn import datasets
# pacote pipeline para combinar preparação e modelagem
from sklearn.pipeline import Pipeline, make_pipeline
# arsenal de preparação
from sklearn.preprocessing import MinMaxScaler # rescala em min-max
from sklearn.preprocessing import StandardScaler # padroniza features removendo média e
#     escalando para variância unitária. Também chamado de z-score
#
from sklearn.neighbors import KNeighborsRegressor as knnr
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
#from sklearn.metrics import root_mean_squared_error as rmse
from sklearn.metrics import make_scorer
#
#    opções de seleção de modelo de entrada
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import KFold as kf
from sklearn.model_selection import StratifiedKFold as skf
from sklearn.model_selection import GridSearchCV as gscv
# pacote para amostragem
from sklearn.utils import resample
#pacotes para apoio a leitura e gravação de datasets
from pathlib import Path
import csv
import pprint
# configurações para os diferentes pacotes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# carga de dados
diabetes = datasets.load_diabetes(as_frame=True)
print(diabetes.DESCR)

In [ ]:
diabetes.frame

In [ ]:
# separação em features e target e profiling do dataset
X = diabetes.data
y = diabetes.target
Xy = diabetes.frame
#relatorio = PR(Xy, title="Diabetes data set")
#relatorio.to_notebook_iframe()


In [ ]:
# configurações para experimentos

n_vizinhos = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 29]#, 49, 109, 259]

k_splits = 10


## Execuções com separação em treino e teste

In [ ]:
# separação em treino e teste, e X e y

treino_X, teste_X, treino_y, teste_y = tts(X, y, random_state=0, test_size=0.2)

print(treino_X.shape)
print(treino_y.shape)
print(teste_X.shape)
print(teste_y.shape)

In [ ]:
# indução do modelo para diferentes números de vizinhos

MAEs = []
RMSEs = []

for n in n_vizinhos:
    modelo = knnr(n_neighbors=n)
    modelo.fit(treino_X, treino_y)
    teste_pred_y = modelo.predict(teste_X)

    MAE = mae(teste_y, teste_pred_y)
    RMSE = mse(teste_y, teste_pred_y, squared=False)
    print('k=', n,'  MAE=', MAE, '  RMSE=', RMSE)

    MAEs.append(MAE)
    RMSEs.append(RMSE)

# colocar num gráfico os erros por número de vizinhos

# No eixo x teremos o número de vizinhos e no eixo y o MAE e RMSE
plt.plot(n_vizinhos, MAEs, '-o')
plt.plot(n_vizinhos, RMSEs, '-x')
# Definindo que no eixo x queremos exibir cada numero de vizinhos
plt.legend(['MAE','RMSE'])
plt.xlabel('Num. vizinhos')
plt.ylabel('Erros')
plt.xticks(n_vizinhos);

### Experimento com reescala de valores, embaralhamento e busca exaustiva nos parâmetros

In [ ]:
# protocolo experimental com preparação e modelagem

modelo = Pipeline(steps=[
     ('modelagem', knnr())
    ])
parametros = {'modelagem__n_neighbors':n_vizinhos}#, 'modelagem__weights':['distance', 'uniform']}
kfold = kf(n_splits=k_splits, shuffle=True, random_state=0)
grade = gscv(modelo, param_grid=parametros, cv=kfold, scoring=make_scorer(mse, greater_is_better=False), return_train_score=True)
grade.fit(treino_X,treino_y)
treino_pred_y = grade.predict(treino_X)
teste_pred_y = grade.predict(teste_X)

n_neighbors_best = grade.best_params_['modelagem__n_neighbors']
#weights_best = grade.best_params_['modelagem__weights']

MAE_treino = mae(treino_y, treino_pred_y)
RMSE_treino = mse(treino_y, treino_pred_y, squared=False)

MAE_teste = mae(teste_y, teste_pred_y)
RMSE_teste = mse(teste_y, teste_pred_y, squared=False)

print('Melhor k:{}  score:{:0.3f}'.format(grade.best_params_, math.sqrt(abs(grade.best_score_))))
print('Escore do modelo com melhores parâmetros:{:0.3f}'.format(math.sqrt(abs(grade.score(teste_X,teste_y)))))

print('k={}  MAE treino={:0.3f}  MAE teste={:0.3f}  RMSE treino={:0.3f}  RMSE teste={:0.3f}'.format(n_neighbors_best, MAE_treino, MAE_teste, RMSE_treino, RMSE_teste))


In [ ]:
grade.best_estimator_

In [ ]:
resultados = pd.DataFrame(grade.cv_results_).sort_values(by='rank_test_score')
resultados.head(20)

### Tabulação de escores de treino e de validação

In [ ]:
# capturar os resultados
n_vizinhos_x = resultados['param_modelagem__n_neighbors'].to_list()
MSE_treino = np.sqrt(np.abs(resultados['mean_train_score'])).sort_index()
MSE_validacao = np.sqrt(np.abs(resultados['mean_test_score'])).sort_index()
# No eixo x teremos o número de vizinhos e no eixo y o MAE e RMSE
plt.plot(n_vizinhos, MSE_treino, '-o')
plt.plot(n_vizinhos, MSE_validacao, '-x')
# Definindo que no eixo x queremos exibir cada numero de vizinhos
plt.legend(['MSE treino','MSE validação'])
plt.xlabel('Num. vizinhos')
plt.ylabel('Escores')
plt.xticks(n_vizinhos);
plt.show()